In [1]:
import sys
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import collections
from annoy import AnnoyIndex
import os
import copy

In [2]:
debug=False
doTest=False
bucketSize=20
minValueOfNrHitsForParticleWithMostHits=5

In [3]:
# pt orice proiect fisier de input din care sa iau date si fisier de output in care sa pun rezultatul obtinut
inputFolderName="/Users/luizaadelinaciucu/Work/ATLAS/data/TrackML/ttbar_mu200-generic"
outputFolderName="/Users/luizaadelinaciucu/Work/ATLAS/TrackML/output_min5"

In [4]:
def get_eventNumber_from_fileName(fileName):
    eventNumber=fileName.replace("event","").replace("-hits.csv","")
    return eventNumber

In [5]:
# calculate event numbers from my folder
# lista goala in care sa pun numerele evenimentelor
list_eventNumber=[]
# The sorted() function returns a sorted list of the specified iterable object 
# Strings are sorted alphabetically, and numbers are sorted numerically
# os operating system 
# hits ca sa nu am de doura ori evenimentul in lista
for fileName in sorted(os.listdir(inputFolderName)):
    if fileName.endswith("-hits.csv"):
        #print(fileName)
        eventNumber=get_eventNumber_from_fileName(fileName)
        #print(eventNumber)
        list_eventNumber.append(eventNumber)
# done for loop
#list_eventNumber=["000000007","000000008"]
print("All events available in my folder. list_eventNumber", list_eventNumber)
    

All events available in my folder. list_eventNumber ['000000000', '000000001', '000000002', '000000003', '000000004', '000000005', '000000006', '000000007', '000000008', '000000009', '000000010', '000000011', '000000012', '000000013', '000000014', '000000015', '000000016', '000000017', '000000018', '000000019', '000000020', '000000021', '000000022', '000000023', '000000024', '000000025', '000000026', '000000027', '000000028', '000000029', '000000030', '000000031', '000000032', '000000033', '000000034', '000000035', '000000036', '000000037', '000000038', '000000039', '000000040', '000000041', '000000042', '000000043', '000000044', '000000045', '000000046', '000000047', '000000048', '000000049', '000000050', '000000051', '000000052', '000000053', '000000054', '000000055', '000000056', '000000057', '000000058', '000000059', '000000060', '000000061', '000000062', '000000063', '000000064', '000000065', '000000066', '000000067', '000000068', '000000069', '000000070', '000000071', '000000072'

In [6]:
# this runs only once per event
def buildAnnoyIndex(nparray_position,metric="angular",ntrees=10,debug=False): 
    
    numberDimension=nparray_position.shape[1] # 3 (x,y,z)
    if debug:
        print("numberDimension",numberDimension,"metric",metric)
    index=AnnoyIndex(numberDimension,metric)
    if debug:
        print("type(index)",type(index))
        print("enumerate data")
    # add each hit to the index
    for i,position in enumerate(nparray_position):
        if debug:
            print("i",i,"position",position)
        index.add_item(i,position)
    # done for loop over hits
    # build the index with 10 trees
    index.build(ntrees) # 10 trees
    return index
# done function

In [7]:
def get_input_output_from_df_hits(df_hits,index):
    list_nparray_input=[]
    list_nparray_output=[]
    
    nparray_volume_id=df_hits["volume_id"].values
    nparray_layer_id=df_hits["layer_id"].values
    
    counterHitForBucket=0
    counterBucket=0
    
    for i in range(len(df_hits)):
        if (
            (nparray_volume_id[i]==8 and nparray_layer_id[i]==2) 
            #or (nparray_volume_id[i]==7 and nparray_layer_id[i]==2)
            #or (nparray_volume_id[i]==9 and nparray_layer_id[i]==14)
            #or (nparray_volume_id[i]==16 and nparray_layer_id[i]==2)
            #or (nparray_volume_id[i]==12 and nparray_layer_id[i]==2)
            #or (nparray_volume_id[i]==18 and nparray_layer_id[i]==12)
            #or (nparray_volume_id[i]==14 and nparray_layer_id[i]==12)
            #or (nparray_volume_id[i]==17 and nparray_layer_id[i]==4)
            )==False:
            continue

        counterHitForBucket+=1
        
        #if (counterHitForBucket%3==1)==False:
            #continue
            
        counterBucket+=1
        
        if debug:
            print("i",i,"counterHitForBucket",counterHitForBucket,"counterBucket",counterBucket)
            
        # using annoy to find the 20 nearest neighboring hits by angle to this hit
        list_index=index.get_nns_by_item(i,bucketSize)
        if debug:
            print("list_index",list_index)
            
        # create bucket
        df_bucket=df_hits.iloc[list_index]
        
        # create NN input for this one bucket
        nparray_input=df_bucket[["x","y","z"]].values.flatten()
        if debug:
            print("nparray_input",nparray_input,"shape",nparray_input.shape,"type",type(nparray_input))
            
        # create NN output for this one bucket
        # identify particle with the largest number of hits in this bucket
        nparray_particleID=df_bucket["particle_id"].values

        if debug:
            print("nparray_particleID",nparray_particleID,"shape",nparray_particleID.shape,"type",type(nparray_particleID))
        dict_particleID_counterParticleID={}
        for particleID in nparray_particleID:
            if particleID not in dict_particleID_counterParticleID:
                dict_particleID_counterParticleID[particleID]=1
            else:
                dict_particleID_counterParticleID[particleID]+=1
        if debug:
            print("dict_particleID_counterParticleID",dict_particleID_counterParticleID)
          
        # find the maximum value of the counters
        particleIDWithMaxHits=0
        counterParticleIDWithMaxHits=0
        for particleID in dict_particleID_counterParticleID:
            counterParticleID=dict_particleID_counterParticleID[particleID]
            if counterParticleID>counterParticleIDWithMaxHits:
                counterParticleIDWithMaxHits=counterParticleID
                particleIDWithMaxHits=particleID
        if debug:
            print("counterBucket",counterBucket,"particleIDWithMaxHits",particleIDWithMaxHits,"counterParticleIDWithMaxHits",counterParticleIDWithMaxHits)        
        
        # create nparray_output
        list_output=[]
        
        # loop over every hit in the bucket
        for particleID in nparray_particleID:      
            if counterParticleIDWithMaxHits<minValueOfNrHitsForParticleWithMostHits:
                list_output.append(-1)
            else:
                # do normally
                if particleID==particleIDWithMaxHits:
                    list_output.append(1)
                else:
                    list_output.append(-1)
                # done if
            # done if
        # done for loop for each hit in the bucket
        if debug:        
            print("list_output",list_output)
        nparray_output=np.array(list_output)
        if debug:
            print("nparray_output",nparray_output)
            
        if debug and doTest:
            print("i",i,"counterBucket",counterBucket)
            plt.plot(df_bucket.x,df_bucket.y,"o",color="red")
            plt.plot(0,0,"r+")
            plt.title(str(i))
            plt.show()
            plotBucketFileNameStem=outputFolderName+"/bucket_i_"+str(i)+"_counterBucket_"+str(counterBucket)+"_eventNumber_"+eventNumber
            # plt.savefig(plotBucketFileNameStem+".png")
            # plt.savefig(plotBucketFileNameStem+".pdf")
        # done if
            
        # add for this current bucket to the list for the entire event (which has several buckets)
        list_nparray_input.append(nparray_input)
        list_nparray_output.append(nparray_output)
    # done for loop over hits in the event
    
    # transform list of numpy array into numpy array of dimension 2 
    # rows are buckets, columns are intput and output values
    nparray_input_all=np.array(list_nparray_input)
    nparray_output_all=np.array(list_nparray_output)
    
    return nparray_input_all, nparray_output_all
# done function      

In [8]:
counter_event_Train=0
counter_event_Test=0
# for loop over eventNumber from list_eventNumber
for i,eventNumber in enumerate(list_eventNumber):
    if i>9:
        continue
    print("i",i,"eventNumber",eventNumber)
    
    inputFileName_hits_recon=inputFolderName+"/event"+eventNumber+"-hits.csv"
    inputFileName_hits_truth=inputFolderName+"/event"+eventNumber+"-truth.csv"
    
    df_hits_recon=pd.read_csv(inputFileName_hits_recon)
    df_hits_truth=pd.read_csv(inputFileName_hits_truth)
    df_hits=pd.concat([df_hits_recon,df_hits_truth],axis=1,sort=False)
    
    # build annoy index
    nparray_position=df_hits[["x","y","z"]].values
    if debug:
        print("nparray_position",nparray_position)

    index=buildAnnoyIndex(nparray_position,metric="angular",ntrees=10,debug=False)
    
    # for loop peste hituri peste hiturile preselectate volume_id=8, layer_id=2
    nparray_input_all,nparray_output_all=get_input_output_from_df_hits(df_hits,index)
    if debug:
        print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all)) 
        print("nparray_output_all",nparray_output_all,"shape",nparray_output_all.shape,"type",type(nparray_output_all))
    
    # reshape only input by adding one extra dimension needed by tensorflow, in practice it puts all into one extra bracket
    nparray_input_all=nparray_input_all.reshape(nparray_input_all.shape[0],nparray_input_all.shape[1],1)
    if debug:
        print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all))

    # calcualte the rest of the event index to division by 10

    if doTest:
        # use two events, one in train, one in test
        nbDenominator=2
        maxRestForTrain=0
    else:
        # running fully
        #nbDenominator=100
        #maxRestForTrain=69
        nbDenominator=10
        maxRestForTrain=6
    # done if
    rest=i%nbDenominator
    if rest<=maxRestForTrain: 
        # put this event into Train: rest=0,1,2,3,4,5,6
        counter_event_Train+=1
        if counter_event_Train==1:
            # it is the first event of type Train, so we simply deep copy it to the output
            nparray_Input_Train_all_Events=copy.deepcopy(nparray_input_all)
            nparray_Output_Train_all_Events=copy.deepcopy(nparray_output_all)
        else:
            # it is not the first, so we concatentane to the new output
            nparray_Input_Train_all_Events=np.concatenate((nparray_Input_Train_all_Events,nparray_input_all),axis=0,out=None)
            nparray_Output_Train_all_Events=np.concatenate((nparray_Output_Train_all_Events,nparray_output_all),axis=0,out=None)
        # done if
    else:
        # put this event into Test: rest=7,8,9
        counter_event_Test+=1
        if counter_event_Test==1:
            # it is the first event of type Test, so we simply deep copy it to the output
            nparray_Input_Test_all_Events=copy.deepcopy(nparray_input_all)
            nparray_Output_Test_all_Events=copy.deepcopy(nparray_output_all)
        else:
            # it is not the first, so we concatentane to the new output
            nparray_Input_Test_all_Events=np.concatenate((nparray_Input_Test_all_Events,nparray_input_all),axis=0,out=None)
            nparray_Output_Test_all_Events=np.concatenate((nparray_Output_Test_all_Events,nparray_output_all),axis=0,out=None)
        # done if
    # done if
# done for loop over eventNumber

eventNumber="all"
fileNameNNInputTrainAll=outputFolderName+"/NN_2_data_Input_Train_"+eventNumber+".npy"
fileNameNNInputTestAll=outputFolderName+"/NN_2_data_Input_Test_"+eventNumber+".npy"
fileNameNNOutputTrainAll=outputFolderName+"/NN_2_data_Output_Train_"+eventNumber+".npy"
fileNameNNOutputTestAll=outputFolderName+"/NN_2_data_Output_Test_"+eventNumber+".npy"

np.save(fileNameNNInputTrainAll,nparray_Input_Train_all_Events)
np.save(fileNameNNOutputTrainAll,nparray_Output_Train_all_Events)
np.save(fileNameNNInputTestAll,nparray_Input_Test_all_Events)
np.save(fileNameNNOutputTestAll,nparray_Output_Test_all_Events)

print("nparray_Input_Train_all_Events",nparray_Input_Train_all_Events,"shape",nparray_Input_Train_all_Events.shape,"type",type(nparray_Input_Train_all_Events))
print("nparray_Input_Test_all_Events",nparray_Input_Test_all_Events,"shape",nparray_Input_Test_all_Events.shape,"type",type(nparray_Input_Test_all_Events))
print("nparray_Output_Train_all_Events",nparray_Output_Train_all_Events,"shape",nparray_Output_Train_all_Events.shape,"type",type(nparray_Output_Train_all_Events))
print("nparray_Output_Test_all_Events",nparray_Output_Test_all_Events,"shape",nparray_Output_Test_all_Events.shape,"type",type(nparray_Output_Test_all_Events))

print("Done all!")

i 0 eventNumber 000000000
i 1 eventNumber 000000001
i 2 eventNumber 000000002
i 3 eventNumber 000000003
i 4 eventNumber 000000004
i 5 eventNumber 000000005
i 6 eventNumber 000000006
i 7 eventNumber 000000007
i 8 eventNumber 000000008
i 9 eventNumber 000000009
nparray_Input_Train_all_Events [[[  -32.3306 ]
  [   -4.28872]
  [ -462.092  ]
  ...
  [  -89.3    ]
  [  -15.1782 ]
  [-1302.5    ]]

 [[  -30.0484 ]
  [  -10.8162 ]
  [ -427.981  ]
  ...
  [  -47.7968 ]
  [  -14.5214 ]
  [ -702.     ]]

 [[  -30.4775 ]
  [   -9.58902]
  [ -421.256  ]
  ...
  [ -112.212  ]
  [  -40.1146 ]
  [-1498.     ]]

 ...

 [[  -31.0392 ]
  [    7.97939]
  [  451.233  ]
  ...
  [  -57.8866 ]
  [   16.1856 ]
  [  822.     ]]

 [[  -30.7311 ]
  [   13.4412 ]
  [  445.29   ]
  ...
  [  -84.0347 ]
  [   39.9226 ]
  [ 1298.     ]]

 [[  -30.6693 ]
  [   14.5378 ]
  [  462.772  ]
  ...
  [  -61.4119 ]
  [   29.2769 ]
  [  962.5    ]]] shape (74955, 60, 1) type <class 'numpy.ndarray'>
nparray_Input_Test_all_Events

In [9]:
print("nparray_Input_Train_all_Events",nparray_Input_Train_all_Events,"shape",nparray_Input_Train_all_Events.shape,"type",nparray_Input_Train_all_Events.dtype)

nparray_Input_Train_all_Events [[[  -32.3306 ]
  [   -4.28872]
  [ -462.092  ]
  ...
  [  -89.3    ]
  [  -15.1782 ]
  [-1302.5    ]]

 [[  -30.0484 ]
  [  -10.8162 ]
  [ -427.981  ]
  ...
  [  -47.7968 ]
  [  -14.5214 ]
  [ -702.     ]]

 [[  -30.4775 ]
  [   -9.58902]
  [ -421.256  ]
  ...
  [ -112.212  ]
  [  -40.1146 ]
  [-1498.     ]]

 ...

 [[  -31.0392 ]
  [    7.97939]
  [  451.233  ]
  ...
  [  -57.8866 ]
  [   16.1856 ]
  [  822.     ]]

 [[  -30.7311 ]
  [   13.4412 ]
  [  445.29   ]
  ...
  [  -84.0347 ]
  [   39.9226 ]
  [ 1298.     ]]

 [[  -30.6693 ]
  [   14.5378 ]
  [  462.772  ]
  ...
  [  -61.4119 ]
  [   29.2769 ]
  [  962.5    ]]] shape (74955, 60, 1) type float64


In [10]:
print("nparray_Output_Train_all_Events",nparray_Output_Train_all_Events,"shape",nparray_Output_Train_all_Events.shape,"type",nparray_Output_Train_all_Events.dtype)

nparray_Output_Train_all_Events [[-1  1 -1 ... -1 -1 -1]
 [ 1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 ...
 [ 1  1  1 ... -1 -1 -1]
 [-1 -1 -1 ...  1  1  1]
 [ 1  1  1 ... -1 -1 -1]] shape (74955, 20) type int64
